# IMDb

เว็บ IMDb รวบรวมข้อมูลเกี่ยวกับภาพยนตร์ต่างๆ และตัดข้อมูลบางส่วนมาให้ใช้ทำการทดลองต่างๆ ได้ https://www.imdb.com/interfaces/

โจทย์นี้คัดกรอง [ข้อมูลบางส่วนใน IMDb](https://drive.google.com/file/d/1V5VNg1WTMTS_eEOQqylektM9ZwdG1uq2/view?usp=sharing) มา โดยโดยมีไฟล์นี้อยู่

**Cast.tsv**
- tconst (string) - alphanumeric unique identifier of the title
- ordering (integer) – a number to uniquely identify rows for a given titleId
- nconst (string) - alphanumeric unique identifier of the name/person
- category (string) - the category of job that person was in
- job (string) - the specific job title if applicable, else '\N'
- characters (string) - the name of the character played if applicable, else '\N'

**Title.tsv**
- tconst (string) - alphanumeric unique identifier of the title
- titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
- originalTitle (string) - original title, in the original language
- isAdult (boolean) - 0: non-adult title; 1: adult title
- startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
- endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
- runtimeMinutes – primary runtime of the title, in minutes
- genres (string array) – includes up to three genres associated with the title

**Star.tsv**
- nconst (string) - alphanumeric unique identifier of the name/person
- primaryName (string)– name by which the person is most often credited
- birthYear – in YYYY format
- deathYear – in YYYY format if applicable, else '\N'
- primaryProfession (array of strings)– the top-3 professions of the person
- knownForTitles (array of tconsts) – titles the person is known for

## Preprocess

ใช้ encoding='utf-8' ตอนอ่านไฟล์

In [9]:
def read_table(fileName:str):
    with open(fileName) as file:
        rawDatas = file.read().strip().split('\n')

    rawDatas = [rawData for rawData in rawDatas if rawData != '']
    return [dict(zip(rawDatas[0].split('	'),rawData.split('	'))) for rawData in rawDatas[1:]]

In [10]:
df_star = read_table('Star.tsv')
df_cast = read_table('Cast.tsv')
df_title = read_table('Title.tsv')

In [11]:
df_title

[{'tconst': 'tt0192789',
  'titleType': 'movie',
  'primaryTitle': 'While Supplies Last',
  'originalTitle': 'While Supplies Last',
  'isAdult': '0',
  'startYear': '2002',
  'endYear': '\\N',
  'runtimeMinutes': '120',
  'genres': 'Comedy,Musical'},
 {'tconst': 'tt4914592',
  'titleType': 'movie',
  'primaryTitle': 'Electric Heart',
  'originalTitle': 'Electric Heart',
  'isAdult': '0',
  'startYear': '2017',
  'endYear': '\\N',
  'runtimeMinutes': '75',
  'genres': 'Adventure,Drama,Music'},
 {'tconst': 'tt4999994',
  'titleType': 'movie',
  'primaryTitle': 'Rain Doll',
  'originalTitle': 'Rain Doll',
  'isAdult': '0',
  'startYear': '2016',
  'endYear': '\\N',
  'runtimeMinutes': '115',
  'genres': 'Drama'},
 {'tconst': 'tt2690572',
  'titleType': 'movie',
  'primaryTitle': 'The Blessed Ones',
  'originalTitle': 'Polaris',
  'isAdult': '0',
  'startYear': '2017',
  'endYear': '\\N',
  'runtimeMinutes': '79',
  'genres': 'Drama'},
 {'tconst': 'tt1562859',
  'titleType': 'movie',
  'pr

# 1. มีภาพยนตร์กี่ประเภท (genres) แต่ละประเภทมีอย่างละกี่เรื่อง (เรียงตาม genre)
หาด้วย Title

```
Action 733
Adult 56
Adventure 277
Animation 132
Biography 17
8Comedy 1694
Crime 488
Documentary 484
Drama 3041
Family 247
Fantasy 218
History 120
Horror 559
Music 128
Musical 74
Mystery 268
News 7
Reality-TV 3
Romance 825
Sci-Fi 181
Sport 80
Thriller 726
War 83
Western 22
\N 270
```

In [12]:
genreData = {}

for title in df_title:
    genres = title['genres'].split(',')
    for genre in genres:
        try:
            genreData[genre]+=1
        except:
            genreData.update({genre:1})

In [13]:
genreData

{'Comedy': 1694,
 'Musical': 74,
 'Adventure': 277,
 'Drama': 3041,
 'Music': 128,
 'Action': 733,
 'Horror': 559,
 'Romance': 825,
 'Sci-Fi': 181,
 'Crime': 488,
 'Documentary': 484,
 'Fantasy': 218,
 'Thriller': 726,
 'Family': 247,
 'Mystery': 268,
 '\\N': 270,
 'War': 83,
 'Biography': 178,
 'Adult': 56,
 'Sport': 80,
 'Western': 22,
 'History': 120,
 'Animation': 132,
 'News': 7,
 'Reality-TV': 3}

Action              733
Adult               56
Adventure           277
Animation           132
Biography           178
Comedy              1694
Crime               488
Documentary         484
Drama               3041
Family              247
Fantasy             218
History             120
Horror              559
Music               128
Musical             74
Mystery             268
News                7
Reality-TV          3
Romance             825
Sci-Fi              181
Sport               80
Thriller            726
War                 83
Western             22
\N                  270


# 2. มีนักแสดงชายและหญิงอย่างละกี่คน
นับจากไฟล์ Star

```
actress 3537
actor 4300
```

In [20]:
genderData = {}

for star in df_star:
    genders = star['primaryProfession'].split(',')
    for gender in genders:
        if gender != 'actor' and gender != 'actress':
            continue
        try:
            genderData[gender] += 1
        except:
            genderData.update({gender:1})

In [21]:
for key, val in genderData.items():
    print(f"{key:<20}{val}")

actress             3537
actor               4300


# 3. นักแสดงที่ยังมีชีวิตมีอายุเท่าไหร่บ้าง แต่ละช่วงมีกี่คน 
หาด้วย Star

```
32 213
33 233
34 288
35 328
36 317
37 365
38 389
39 438
40 430
41 448
42 495
43 460
44 442
45 463
46 472
47 447
48 505
49 511
50 498
```

In [29]:
filteredLife = list(filter(lambda star:star['deathYear'] == '\\N',df_star))

In [30]:
ageData = {}

for life in filteredLife:
    try:
        ageData[2021-int(life['birthYear'])]+=1
    except:
        ageData.update({2021-int(life['birthYear']):1})
    

In [31]:
ageData

{47: 447,
 46: 472,
 48: 505,
 42: 495,
 49: 511,
 50: 498,
 40: 430,
 41: 448,
 45: 463,
 44: 442,
 39: 438,
 43: 460,
 38: 389,
 35: 328,
 34: 288,
 36: 317,
 33: 233,
 37: 365,
 32: 213}

# 4. มีนักแสดงกี่คนที่เคยแสดงในภาพยนตร์ Action 
หาด้วย Cast + Title

```
992
```

In [40]:
action_movies = []

for movie in df_title:
    genres = movie['genres'].split(',')

    try:
        genres.index('Action')
        action_movies.append(movie['tconst'])
    except:
        continue

In [47]:
actor_action = set()

for cast in df_cast:
    try:
        action_movies.index(cast['tconst'])
        if cast['category'] != 'actor' and cast['category'] != 'actress':
            continue
        actor_action.add(cast['nconst'])
    except:
        continue


In [48]:
len(actor_action)

1066

# 5. มีนักแสดงกี่คนที่แสดงในภาพยนตร์มากกว่า 1 ประเภท 
หาด้วย Cast + Title

```
3861
```

3861

# 6. มีผู้กำกับกี่คนที่เป็นนักแสดงด้วย 
หาด้วย Star

```
1424
```

1424

# 7. หาจำนวนภาพยนต์แนวรอมคอมในแต่ละปี (นับเฉพาะปีที่มีตั้งแต่ 1 เรื่องขึ้นไป)
หาด้วย Title

```
1991 3
1992 2
1993 3
1994 1
1995 4
1996 8
1997 3
1998 13
1999 8
2000 8
2001 7
2002 15
2003 9
2004 8
2005 15
2006 13
2007 15
2008 9
2009 10
2010 25
2011 24
2012 18
2013 14
2014 24
2015 21
2016 22
2017 10
2018 14
2019 15
2020 5
```

1991 3
1992 2
1993 3
1994 1
1995 4
1996 8
1997 3
1998 13
1999 8
2000 8
2001 7
2002 15
2003 9
2004 8
2005 15
2006 13
2007 15
2008 9
2009 10
2010 25
2011 24
2012 18
2013 14
2014 24
2015 21
2016 22
2017 10
2018 14
2019 15
2020 5
